In [2]:
import pandas as pd
import math
from ast import literal_eval

In [3]:

orig = pd.read_csv("../raw/usa_with_details.csv")
orig_n = orig.shape[0]
# todo : scams => company name , //orgs , ?// wfh single Node , //states , ? country names mentioned.

In [14]:
cnt = []
for i in range(orig_n):
    if not isinstance(orig["Description"][i],str):
        orig["Description"][i] = ""
        cnt.append(i)
cnt

/home/sakshamm/environments/my_env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


[5222, 16686]

In [9]:
import spacy
spacy.load('en_core_web_sm')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens


In [10]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to /home/sakshamm/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [11]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sakshamm/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [43]:
import random
text_data = []
for fr in range(orig_n):
    line = orig["Description"][fr]
    tokens = prepare_text_for_lda(line)
    if random.random() > .90:
        print(fr)
        text_data.append(tokens)

3
21
27
40
41
44
54
59
79
89
93
96
102
104
105
110
117
137
147
166
170
205
213
220
229
230
251
266
279
284
314
332
335
337
349
372
373
376
403
430
439
465
478
479
493
498
516
534
538
565
585
610
621
637
642
644
649
654
658
669
676
682
688
690
694
699
705
712
716
717
761
795
799
819
832
842
843
847
850
854
865
880
905
944
947
978
979
1011
1024
1038
1053
1057
1069
1082
1086
1111
1124
1139
1163
1169
1172
1179
1189
1203
1217
1219
1222
1228
1255
1262
1292
1308
1321
1327
1364
1376
1388
1395
1404
1407
1415
1418
1425
1443
1461
1467
1471
1483
1500
1509
1519
1521
1536
1541
1545
1553
1569
1576
1592
1593
1610
1623
1634
1652
1660
1675
1688
1690
1713
1725
1729
1739
1740
1746
1747
1756
1773
1786
1789
1795
1805
1816
1819
1824
1837
1841
1884
1905
1915
1932
1939
1954
1957
1962
1966
1968
1969
2008
2009
2011
2013
2021
2035
2038
2039
2050
2055
2059
2062
2071
2073
2079
2093
2104
2122
2127
2129
2152
2154
2159
2185
2193
2200
2212
2221
2223
2231
2280
2282
2293
2294
2303
2325
2342
2362
2376
2396
2417
2421
2427


16095
16104
16114
16116
16130
16138
16151
16154
16168
16172
16180
16187
16191
16192
16193
16194
16202
16214
16228
16232
16234
16243
16252
16255
16265
16268
16272
16273
16293
16311
16319
16327
16333
16338
16349
16356
16371
16373
16376
16408
16415
16416
16435
16437
16441
16450
16456
16464
16477
16492
16508
16520
16522
16549
16554
16569
16596
16598
16602
16627
16638
16670
16671
16679
16691
16703
16715


In [44]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [48]:
import gensim
NUM_TOPICS = 10
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.016*"email" + 0.009*"description" + 0.009*"phone" + 0.005*"contact"')
(1, '0.024*"company" + 0.022*"would" + 0.016*"check" + 0.016*"ask"')
(2, '0.042*"check" + 0.023*"assignment" + 0.022*"cards" + 0.021*"store"')
(3, '0.026*"company" + 0.019*"receive" + 0.018*"package" + 0.011*"email"')
(4, '0.040*"email" + 0.022*"receive" + 0.021*"would" + 0.017*"company"')
(5, '0.020*"training" + 0.014*"amazon" + 0.013*"online" + 0.011*"resume"')
(6, '0.016*"position" + 0.016*"company" + 0.008*"information" + 0.008*"interview"')
(7, '0.013*"position" + 0.010*"please" + 0.010*"project" + 0.009*"period"')
(8, '0.055*"check" + 0.028*"email" + 0.017*"receive" + 0.016*"deposit"')
(9, '0.018*"company" + 0.013*"working" + 0.011*"position" + 0.010*"check"')


In [49]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)